In [1]:
from pathlib import Path
import pandas as pd
database_folder = 'data/database/'
Path (database_folder+'TorontoRestaurantReviewDatabase.db').touch()

In [2]:
import sqlite3

conn = sqlite3.connect(database_folder+'TorontoRestaurantReviewDatabase.db') #,detect_types=0
c = conn.cursor()

In [3]:
c.description

In [4]:
query = '''SELECT name FROM sqlite_master WHERE type = 'table' ORDER BY name '''

def printSchema(connection):
    connection.execute(query)
    for (tableName,) in connection.fetchall():
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))


printSchema(c)

cuisine:
  0: place_id(TEXT)
  1: name_google(TEXT)
  2: category_tripAdvisor_1(TEXT)
  3: category_tripAdvisor_2(TEXT)
location:
  0: place_id(TEXT)
  1: formatted_address(TEXT)
  2: lattitude(REAL)
  3: longtitude(REAL)
  4: city(TEXT)
  5: postalCode(TEXT)
responses:
  0: review_id(TEXT)
  1: text(TEXT)
  2: place_id(TEXT)
restaurants:
  0: place_id(TEXT)
  1: name_tripAdvisor(TEXT)
  2: rank_tripAdvisor(INTEGER)
  3: name_google(TEXT)
  4: cuisine(TEXT)
  5: retrieve_date(TEXT)
  6: rating_tripAdvisor(TEXT)
  7: rating_google(REAL)
  8: status(TEXT)
  9: review_count(INTEGER)
reviews:
  0: review_id(TEXT)
  1: user_id(TEXT)
  2: text(TEXT)
  3: date_retrieved(TEXT)
  4: date_relative(TEXT)
  5: place_id(TEXT)
web:
  0: place_id(TEXT)
  1: url_googleMaps(TEXT)


In [14]:
query = '''
SELECT rest.place_id,
       rev.text as revText,
       resp.review_id,
       resp.text as respText,
       rest.review_count,
       rest.rating_google,
       cuisine.category_tripAdvisor_1 as resType_1,
       cuisine.category_tripAdvisor_2 as resType_2
FROM   
    reviews as rev 
    INNER JOIN responses as resp ON rev.review_id = resp.review_id
    INNER JOIN restaurants as rest ON rest.place_id = rev.place_id
    INNER JOIN cuisine as cuisine ON cuisine.place_id = rev.place_id

    ; ''' #AND status = 'OPERATIONAL'
# responses:
#   0: review_id(TEXT)
#   1: responded(INTEGER)
#   2: text(TEXT)
#   3: place_id(TEXT)

# 
#        

c.execute(query)
c.fetchall()

[('ChIJedJ0GGI2K4gR6dco_CjRntY',
  "The calzone was tasty! Felt a bit heavy on the bread portion but If you order extra sauce, It's useful for dipping it in. They charged $3 for mushrooms to be added, which is a bit higher than the standard. The wine sangria fell a bit flat …",
  'ChZDSUhNMG9nS0VJQ0FnSUNheHZyeFN3EAE',
  'Thank you for your feedback. We appreciate it and will share with staff. We do hope you will join us again sometime.',
  902,
  4.3,
  'Italian',
  'Pizza'),
 ('ChIJedJ0GGI2K4gR6dco_CjRntY',
  'By far, the best pizza I’ve ever had in Canada. We ordered the Spettacolare and it was really good. The staff is amazing, fresh ingredients and orders are super quick. Bretta was our server at Old Mill Dr, and 5 stars to her. Courteous and attentive.  We will be definitely coming back.',
  'ChZDSUhNMG9nS0VJQ0FnSUNxMVlUTWZBEAE',
  'Grazie! Please join us again soon!',
  902,
  4.3,
  'Italian',
  'Pizza'),
 ('ChIJedJ0GGI2K4gR6dco_CjRntY',
  'We had the Prosciutto e Rucola last ni

In [6]:
# import pandas as pd
pd_resp = pd.read_sql(query,conn)
len(pd_resp)

60842

In [7]:
pd_resp.columns

Index(['place_id', 'revText', 'review_id', 'respText', 'review_count',
       'rating_google', 'resType_1', 'resType_2'],
      dtype='object')

In [15]:
query = '''
SELECT rest.place_id,
       rev.review_id,
       rev.text as revText,
       rest.review_count,
       rest.rating_google,
       cuisine.category_tripAdvisor_1 as resType_1,
       cuisine.category_tripAdvisor_2 as resType_2
FROM   
    reviews as rev 
    INNER JOIN restaurants as rest ON rest.place_id = rev.place_id
    INNER JOIN cuisine as cuisine ON cuisine.place_id = rev.place_id

    ; ''' #AND status = 'OPERATIONAL'
# responses:
#   0: review_id(TEXT)
#   1: responded(INTEGER)
#   2: text(TEXT)
#   3: place_id(TEXT)

# 
#        

c.execute(query)
c.fetchall()


[('ChIJedJ0GGI2K4gR6dco_CjRntY',
  'ChZDSUhNMG9nS0VJQ0FnSUNheHZyeFN3EAE',
  "The calzone was tasty! Felt a bit heavy on the bread portion but If you order extra sauce, It's useful for dipping it in. They charged $3 for mushrooms to be added, which is a bit higher than the standard. The wine sangria fell a bit flat …",
  902,
  4.3,
  'Italian',
  'Pizza'),
 ('ChIJedJ0GGI2K4gR6dco_CjRntY',
  'ChZDSUhNMG9nS0VJQ0FnSUNxMVlUTWZBEAE',
  'By far, the best pizza I’ve ever had in Canada. We ordered the Spettacolare and it was really good. The staff is amazing, fresh ingredients and orders are super quick. Bretta was our server at Old Mill Dr, and 5 stars to her. Courteous and attentive.  We will be definitely coming back.',
  902,
  4.3,
  'Italian',
  'Pizza'),
 ('ChIJedJ0GGI2K4gR6dco_CjRntY',
  'ChZDSUhNMG9nS0VJQ0FnSUR5c1pUZGJBEAE',
  'We had the Prosciutto e Rucola last night and it’s probably one of the best pizza I have ever had. The cheat night was worth it! Will comeback again soon. …',


In [16]:
pd_rev = pd.read_sql(query,conn)
len(pd_rev)

378261

In [17]:
pd_rev.columns

Index(['place_id', 'review_id', 'revText', 'review_count', 'rating_google',
       'resType_1', 'resType_2'],
      dtype='object')

In [8]:
query = '''
SELECT rest.place_id,
       rest.name_google as name,
       rest.review_count,
       rest.rating_google,
       loc.city,
       cuisine.category_tripAdvisor_1 as resType_1,
       cuisine.category_tripAdvisor_2 as resType_2
FROM   
    restaurants as rest 
    INNER JOIN cuisine as cuisine ON cuisine.place_id = rest.place_id
    INNER JOIN location as loc ON loc.place_id = rest.place_id
    ; ''' #AND status = 'OPERATIONAL'
# responses:
#   0: review_id(TEXT)
#   1: responded(INTEGER)
#   2: text(TEXT)
#   3: place_id(TEXT)

# 
#        

c.execute(query)
c.fetchall()


[('ChIJedJ0GGI2K4gR6dco_CjRntY',
  'Goodfellas Wood Oven Pizza',
  902,
  4.3,
  'York',
  'Italian',
  'Pizza'),
 ('ChIJfQJkpzMyK4gRDnUt3YFXa5w',
  "Moxie's Grill & Bar",
  1634,
  4.0,
  'Toronto',
  'Grill',
  'Bar'),
 ('ChIJ466I-9o2K4gRxzeisSIQUzg',
  'New Orleans Seafood & Steakhouse',
  587,
  4.8,
  'York',
  'Steakhouse',
  'Cajun & Creole'),
 ('ChIJrxCATO8zK4gRHe57JQWul2Q',
  'Scaramouche Restaurant',
  992,
  4.7,
  'Toronto',
  'French',
  'International'),
 ('ChIJryijc9s0K4gRG9aU7SDTXdA',
  'Alo',
  1353,
  4.7,
  'Toronto',
  'French',
  'European'),
 ('ChIJf4hIwkXL1IkR1FE_9q56MKQ',
  'The Tilted Dog Pub & Kitchen',
  404,
  4.5,
  'Toronto',
  'Bar',
  'Pub'),
 ('ChIJHwcmTRk2K4gRm85aUecgRe8',
  'LaVinia Restaurant',
  418,
  4.6,
  'Etobicoke',
  'Mediterranean',
  'Spanish'),
 ('ChIJmzR9dJU0K4gRj44f61gBfss',
  'RASA',
  591,
  4.6,
  'Toronto',
  'Canadian',
  'International'),
 ('ChIJ1ZGZKNk0K4gRaouNzuptWV8', 'PAI', 8817, 4.6, 'Toronto', 'Asian', 'Thai'),
 ('ChIJJZP2IzP

In [9]:
pd_rest = pd.read_sql(query,conn)
len(pd_rest)

2168

In [10]:
pd_rest.head()

,place_id,name,review_count,rating_google,city,resType_1,resType_2
0,ChIJedJ0GGI2K4gR6dco_CjRntY,Goodfellas Wood Oven Pizza,902,4.3,York,Italian,Pizza
1,ChIJfQJkpzMyK4gRDnUt3YFXa5w,Moxie's Grill & Bar,1634,4.0,Toronto,Grill,Bar
2,ChIJ466I-9o2K4gRxzeisSIQUzg,New Orleans Seafood & Steakhouse,587,4.8,York,Steakhouse,Cajun & Creole
3,ChIJrxCATO8zK4gRHe57JQWul2Q,Scaramouche Restaurant,992,4.7,Toronto,French,International
4,ChIJryijc9s0K4gRG9aU7SDTXdA,Alo,1353,4.7,Toronto,French,European


In [60]:
# !pip -m spacy download en_core_web_sm


SyntaxError: invalid syntax (<ipython-input-60-e8a31c0c54ec>, line 1)

### readability

In [11]:
ratingBins = [];
for ind, row in pd_rest.iterrows():
    if (row.rating_google <3) & (row.rating_google>=2):
        ratingBins.append(2)
    elif (row.rating_google <4) & (row.rating_google>=3):
        ratingBins.append(3)
    elif (row.rating_google <5) & (row.rating_google>=4):
        ratingBins.append(4)
    elif (row.rating_google ==5):
        ratingBins.append(5)
    else:
        ratingBins.append(1)
        
pd_rest['ratingBins'] = ratingBins


In [12]:
pd_rest.head()

,place_id,name,review_count,rating_google,city,resType_1,resType_2,ratingBins
0,ChIJedJ0GGI2K4gR6dco_CjRntY,Goodfellas Wood Oven Pizza,902,4.3,York,Italian,Pizza,4
1,ChIJfQJkpzMyK4gRDnUt3YFXa5w,Moxie's Grill & Bar,1634,4.0,Toronto,Grill,Bar,4
2,ChIJ466I-9o2K4gRxzeisSIQUzg,New Orleans Seafood & Steakhouse,587,4.8,York,Steakhouse,Cajun & Creole,4
3,ChIJrxCATO8zK4gRHe57JQWul2Q,Scaramouche Restaurant,992,4.7,Toronto,French,International,4
4,ChIJryijc9s0K4gRG9aU7SDTXdA,Alo,1353,4.7,Toronto,French,European,4


In [18]:
# pd_rev2 = pd_rev#[pd_rev.responded ==1]
print(len(set(pd_rev.place_id)))
# print(len(set(pd_rev.place_id)))
response_rate = []
# rating = []
# ratingBins = []
# resType_1  = []
# resType_2 = []
for rest in pd_rest.place_id.to_list(): #those that are responded
    response_rate.append(len(pd_resp[(pd_resp.place_id == rest)]))
#     rating.append(pd_rest[pd_rest.place_id == rest]['rating_google'].to_list()[0])
#     ratingBins.append(pd_rest[pd_rest.place_id == rest]['ratingBins'].to_list()[0])
#     resType_1.append(pd_rest[pd_rest.place_id == rest]['resType_1'].to_list()[0])
#     resType_2.append(pd_rest[pd_rest.place_id == rest]['resType_2'].to_list()[0])


2122


In [19]:
pd_rest['response_rate'] = response_rate

In [20]:
pd_rest.head()

,place_id,name,review_count,rating_google,city,resType_1,resType_2,ratingBins,response_rate
0,ChIJedJ0GGI2K4gR6dco_CjRntY,Goodfellas Wood Oven Pizza,902,4.3,York,Italian,Pizza,4,202
1,ChIJfQJkpzMyK4gRDnUt3YFXa5w,Moxie's Grill & Bar,1634,4.0,Toronto,Grill,Bar,4,137
2,ChIJ466I-9o2K4gRxzeisSIQUzg,New Orleans Seafood & Steakhouse,587,4.8,York,Steakhouse,Cajun & Creole,4,1
3,ChIJrxCATO8zK4gRHe57JQWul2Q,Scaramouche Restaurant,992,4.7,Toronto,French,International,4,0
4,ChIJryijc9s0K4gRG9aU7SDTXdA,Alo,1353,4.7,Toronto,French,European,4,0


In [21]:
# create a body of text for reviews of each restaurant
# revDict = dict()
allRevList = []

for place in pd_rest.place_id:
    revList = pd_rev[pd_rev.place_id == place]['revText'].to_list()
    revList = [x for x in revList if x != None]
    allRevList.append(' '.join(revList))

    
pd_rest['all_reviews_Text'] = allRevList

In [22]:
# len(allRevList)
# allRevList[10]
# del allRevList
pd_rest.head()

,place_id,name,review_count,rating_google,city,resType_1,resType_2,ratingBins,response_rate,all_reviews_Text
0,ChIJedJ0GGI2K4gR6dco_CjRntY,Goodfellas Wood Oven Pizza,902,4.3,York,Italian,Pizza,4,202,The calzone was tasty! Felt a bit heavy on the...
1,ChIJfQJkpzMyK4gRDnUt3YFXa5w,Moxie's Grill & Bar,1634,4.0,Toronto,Grill,Bar,4,137,We went there on a week day and our group of 5...
2,ChIJ466I-9o2K4gRxzeisSIQUzg,New Orleans Seafood & Steakhouse,587,4.8,York,Steakhouse,Cajun & Creole,4,1,This unassuming restaurant is so good! The ser...
3,ChIJrxCATO8zK4gRHe57JQWul2Q,Scaramouche Restaurant,992,4.7,Toronto,French,International,4,0,Ate there last night as a goodbye meal from To...
4,ChIJryijc9s0K4gRG9aU7SDTXdA,Alo,1353,4.7,Toronto,French,European,4,0,Well it’s Alo. French inspired Canadian fine d...


In [23]:
del allRevList

In [24]:
# create a body of text for responses from each restaurant

allRespList = []

for p,place in enumerate(pd_rest.place_id.to_list()):
    respList = pd_resp[pd_resp.place_id == place]['respText'].to_list()
    respList = [x for x in respList if x != None]
    allRespList.append(' '.join(respList))
#     print(p)

    
pd_rest['all_resps_Text'] = allRespList

In [25]:
pd_rest.head()

,place_id,name,review_count,rating_google,city,resType_1,resType_2,ratingBins,response_rate,all_reviews_Text,all_resps_Text
0,ChIJedJ0GGI2K4gR6dco_CjRntY,Goodfellas Wood Oven Pizza,902,4.3,York,Italian,Pizza,4,202,The calzone was tasty! Felt a bit heavy on the...,Thank you for your feedback. We appreciate it ...
1,ChIJfQJkpzMyK4gRDnUt3YFXa5w,Moxie's Grill & Bar,1634,4.0,Toronto,Grill,Bar,4,137,We went there on a week day and our group of 5...,Thank you Eric for taking the time to share yo...
2,ChIJ466I-9o2K4gRxzeisSIQUzg,New Orleans Seafood & Steakhouse,587,4.8,York,Steakhouse,Cajun & Creole,4,1,This unassuming restaurant is so good! The ser...,"Hi Bibin, I think you may have reviewed the w..."
3,ChIJrxCATO8zK4gRHe57JQWul2Q,Scaramouche Restaurant,992,4.7,Toronto,French,International,4,0,Ate there last night as a goodbye meal from To...,
4,ChIJryijc9s0K4gRG9aU7SDTXdA,Alo,1353,4.7,Toronto,French,European,4,0,Well it’s Alo. French inspired Canadian fine d...,


In [26]:
del allRespList

In [27]:
pip install spacy-readability


  Created wheel for ujson: filename=ujson-1.35-cp39-cp39-win_amd64.whl size=21777 sha256=cc1aedc2cc926a71949c61249b9f0442fd0b8f678b1500a51fba3157f86f71c4
  Stored in directory: c:\users\hajih\appdata\local\pip\cache\wheels\ed\f4\37\aacbb6b0af6a42b379ffdce5a4a70c7c242f1fb71ac975172d
Successfully built ujson
Note: you may need to restart the kernel to use updated packages.


In [31]:
import spacy
from spacy_readability import Readability

nlp = spacy.load('en_core_web_md')
read = Readability()
nlp.add_pipe(read, last=True)


In [32]:
doc = nlp("I am some really difficult text to read because I use obnoxiously large words.")

print(doc._.flesch_kincaid_grade_level)
print(doc._.flesch_kincaid_reading_ease)
print(doc._.dale_chall)
print(doc._.smog)
print(doc._.coleman_liau_index)
print(doc._.automated_readability_index)
print(doc._.forcast)


8.412857142857145
59.68214285714288
7.714471428571429
0
8.96571428571428
7.1014285714285705
0


In [35]:
# get reading ease for each restaurant reviews
# import numpy as np
fleschEaseRev  = []
fleschEaseResp = []

for r,(revs,resps) in enumerate(zip(pd_rest.all_reviews_Text.to_list(),pd_rest.all_resps_Text.to_list())):
    if len(revs) > 0:
        fleschEaseRev.append(nlp(revs)._.flesch_kincaid_reading_ease)
    else:
        fleschEaseRev.append(0)
    if len(resps) > 0:
        fleschEaseResp.append(nlp(resps)._.flesch_kincaid_reading_ease)
    else:
        fleschEaseResp.append(0)

    print (r)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [38]:
pd_rest['fleschEaseReviews'] = fleschEaseRev

In [39]:
pd_rest['fleschEaseResponses'] = fleschEaseResp

In [40]:
pd_rest.head()

,place_id,name,review_count,rating_google,city,resType_1,resType_2,ratingBins,response_rate,all_reviews_Text,all_resps_Text,fleschEaseReviews,fleschEaseResponses
0,ChIJedJ0GGI2K4gR6dco_CjRntY,Goodfellas Wood Oven Pizza,902,4.3,York,Italian,Pizza,4,202,The calzone was tasty! Felt a bit heavy on the...,Thank you for your feedback. We appreciate it ...,70.844264,90.214730
1,ChIJfQJkpzMyK4gRDnUt3YFXa5w,Moxie's Grill & Bar,1634,4.0,Toronto,Grill,Bar,4,137,We went there on a week day and our group of 5...,Thank you Eric for taking the time to share yo...,77.283475,82.156275
2,ChIJ466I-9o2K4gRxzeisSIQUzg,New Orleans Seafood & Steakhouse,587,4.8,York,Steakhouse,Cajun & Creole,4,1,This unassuming restaurant is so good! The ser...,"Hi Bibin, I think you may have reviewed the w...",74.365157,90.935000
3,ChIJrxCATO8zK4gRHe57JQWul2Q,Scaramouche Restaurant,992,4.7,Toronto,French,International,4,0,Ate there last night as a goodbye meal from To...,,71.156440,0.000000
4,ChIJryijc9s0K4gRG9aU7SDTXdA,Alo,1353,4.7,Toronto,French,European,4,0,Well it’s Alo. French inspired Canadian fine d...,,70.884624,0.000000


In [41]:
import joblib

In [42]:
joblib.dump(pd_rest,'pd_rest.jl')

['pd_rest.jl']

In [43]:
xxx = joblib.load('pd_rest.jl')

In [44]:
xxx

,place_id,name,review_count,rating_google,city,resType_1,resType_2,ratingBins,response_rate,all_reviews_Text,all_resps_Text,fleschEaseReviews,fleschEaseResponses
0,ChIJedJ0GGI2K4gR6dco_CjRntY,Goodfellas Wood Oven Pizza,902,4.3,York,Italian,Pizza,4,202,The calzone was tasty! Felt a bit heavy on the...,Thank you for your feedback. We appreciate it ...,70.844264,90.214730
1,ChIJfQJkpzMyK4gRDnUt3YFXa5w,Moxie's Grill & Bar,1634,4.0,Toronto,Grill,Bar,4,137,We went there on a week day and our group of 5...,Thank you Eric for taking the time to share yo...,77.283475,82.156275
2,ChIJ466I-9o2K4gRxzeisSIQUzg,New Orleans Seafood & Steakhouse,587,4.8,York,Steakhouse,Cajun & Creole,4,1,This unassuming restaurant is so good! The ser...,"Hi Bibin, I think you may have reviewed the w...",74.365157,90.935000
3,ChIJrxCATO8zK4gRHe57JQWul2Q,Scaramouche Restaurant,992,4.7,Toronto,French,International,4,0,Ate there last night as a goodbye meal from To...,,71.156440,0.000000
4,ChIJryijc9s0K4gRG9aU7SDTXdA,Alo,1353,4.7,Toronto,French,European,4,0,Well it’s Alo. French inspired Canadian fine d...,,70.884624,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,ChIJ2XZ3tVXQ1IkRtd7R_IGVFHs,Simit the big bagel,22,NaN,Scarborough,Cafe,Turkish,1,0,A taste of Turkey. Best simit bakery in Toront...,,85.153750,0.000000
2164,ChIJP8IxfNs0K4gRPzz0XjSS1V0,Fancy Franks,405,NaN,Toronto,Canadian,Fast food,1,101,"Unfortunately the food is a hit or miss here, ...",Hope to see you again soon!☺️ Thank you so muc...,82.286364,100.100758
2165,ChIJu34dbAxD04kRny8N8yLWSi4,Watermark Restaurant,259,NaN,Niagara Falls,Italian,American,1,32,This restaurant is highly recommended. Went th...,Thank you for your comments. We are glad you e...,79.139954,61.725171
2166,ChIJp1jFXvk1K4gRaKmLr_DxnAQ,Marzitelli,118,NaN,Ottawa,Italian,French,1,12,"Just tried Marzitelli here in Ottawa, which to...","Hi, Thanks for taking the time to review our ...",72.690190,73.481030


In [50]:
# import spacy
# !pip install textstat
# from textstat.textstat import textstatistics,legacy_round

# # Splits the text into sentences, using
# # Spacy's sentence segmentation which can
# # be found at https://spacy.io/usage/spacy-101
# def break_sentences(text):
#     nlp = spacy.load('en_core_web_md')
#     doc = nlp(text)
#     return list(doc.sents)

# # Returns Number of Words in the text
# def word_count(text):
#     sentences = break_sentences(text)
#     words = 0
#     for sentence in sentences:
#         words += len([token for token in sentence])
#     return words

# # Returns the number of sentences in the text
# def sentence_count(text):
#     sentences = break_sentences(text)
#     return len(sentences)

# # Returns average sentence length
# def avg_sentence_length(text):
#     words = word_count(text)
#     sentences = sentence_count(text)
#     average_sentence_length = float(words / sentences)
#     return average_sentence_length

# # Textstat is a python package, to calculate statistics from
# # text to determine readability,
# # complexity and grade level of a particular corpus.
# # Package can be found at https://pypi.python.org/pypi/textstat
# def syllables_count(word):
#     return textstatistics().syllable_count(word)

# # Returns the average number of syllables per
# # word in the text
# def avg_syllables_per_word(text):
#     syllable = syllables_count(text)
#     words = word_count(text)
#     ASPW = float(syllable) / float(words)
#     return legacy_round(ASPW, 1)

# # Return total Difficult Words in a text
# def difficult_words(text):
    
#     nlp = spacy.load('en_core_web_sm')
#     doc = nlp(text)
#     # Find all words in the text
#     words = []
#     sentences = break_sentences(text)
#     for sentence in sentences:
#         words += [str(token) for token in sentence]

#     # difficult words are those with syllables >= 2
#     # easy_word_set is provide by Textstat as
#     # a list of common words
#     diff_words_set = set()
    
#     for word in words:
#         syllable_count = syllables_count(word)
#         if word not in nlp.Defaults.stop_words and syllable_count >= 2:
#             diff_words_set.add(word)

#     return len(diff_words_set)

# # A word is polysyllablic if it has more than 3 syllables
# # this functions returns the number of all such words
# # present in the text
# def poly_syllable_count(text):
#     count = 0
#     words = []
#     sentences = break_sentences(text)
#     for sentence in sentences:
#         words += [token for token in sentence]
    

#     for word in words:
#         syllable_count = syllables_count(word)
#         if syllable_count >= 3:
#             count += 1
#     return count


# def flesch_reading_ease(text):
#     """
#         Implements Flesch Formula:
#         Reading Ease score = 206.835 - (1.015 × ASL) - (84.6 × ASW)
#     Here,
#         ASL = average sentence length (number of words
#                 divided by number of sentences)
#         ASW = average word length in syllables (number of syllables
#                 divided by number of words)
#     """
#     FRE = 206.835 - float(1.015 * avg_sentence_length(text)) -\
#         float(84.6 * avg_syllables_per_word(text))
#     return legacy_round(FRE, 2)


# def gunning_fog(text):
#     per_diff_words = (difficult_words(text) / word_count(text) * 100) + 5
#     grade = 0.4 * (avg_sentence_length(text) + per_diff_words)
#     return grade


# def smog_index(text):
#     """
#     Implements SMOG Formula / Grading
#         SMOG grading = 3 + ?polysyllable count.
#         Here,
#     polysyllable count = number of words of more
#     than two syllables in a sample of 30 sentences.
#     """

#     if sentence_count(text) >= 3:
#         poly_syllab = poly_syllable_count(text)
#         SMOG = (1.043 * (30*(poly_syllab / sentence_count(text)))**0.5) \
#                 + 3.1291
#         return legacy_round(SMOG, 1)
#     else:
#         return 0


# def dale_chall_readability_score(text):
#     """
#         Implements Dale Challe Formula:
#         Raw score = 0.1579*(PDW) + 0.0496*(ASL) + 3.6365
#         Here,
#             PDW = Percentage of difficult words.
#             ASL = Average sentence length
#     """
#     words = word_count(text)
#     # Number of words not termed as difficult words
#     count = word_count - difficult_words(text)
#     if words > 0:

#         # Percentage of words not on difficult word list

#         per = float(count) / float(words) * 100
    
#     # diff_words stores percentage of difficult words
#     diff_words = 100 - per

#     raw_score = (0.1579 * diff_words) + \
#                 (0.0496 * avg_sentence_length(text))
    
#     # If Percentage of Difficult Words is greater than 5 %, then;
#     # Adjusted Score = Raw Score + 3.6365,
#     # otherwise Adjusted Score = Raw Score

#     if diff_words > 5:

#         raw_score += 3.6365
        
#     return legacy_round(score, 2)


In [53]:
# get reading ease for each restaurant reviews
fleschEaseRev  = []
fleschEaseResp = []

for r,(revs,resps) in enumerate(zip(pd_rest.all_reviews_Text.to_list(),pd_rest.all_resps_Text.to_list())):
    if len(revs) > 0:
        fleschEaseRev.append(flesch_reading_ease(revs))
    else:
        fleschEaseRev.append(0)
    if len(resps) > 0:
        fleschEaseResp.append(flesch_reading_ease(resps))
    else:
        fleschEaseResp.append(0)

    print (r)


[94.64,
 93.95,
 94.76,
 94.13,
 93.87,
 95.12,
 95.24,
 94.22,
 93.99,
 94.06,
 96.99,
 95.03,
 94.0,
 94.09,
 94.14,
 94.48,
 95.22,
 94.65,
 93.15,
 93.73,
 94.19,
 94.73,
 102.39,
 95.81,
 94.26,
 102.26,
 93.79,
 94.43,
 94.22,
 102.98,
 94.5,
 93.63,
 94.78,
 94.45,
 93.51,
 94.84,
 94.83,
 93.85,
 94.26,
 94.86,
 95.09]